In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **1. Importing Necessary Libraries**

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree,svm
from sklearn.metrics import accuracy_score


In [3]:
dataset=pd.read_csv('/kaggle/input/titanic/train.csv')
dataset.head(10)

In [4]:
print(dataset.shape)
print('The shape of our training set : %s passengers and %s features'%(dataset.shape[0],dataset.shape[1]))

In [5]:
dataset.info()

In [6]:
#Check for null values
dataset.isnull().sum()

In [7]:
heatmap=sns.heatmap(dataset[['Survived','SibSp','Parch','Age','Fare']].corr(),annot=True)
sns.set(rc={'figure.figsize':(12,10)})

In [8]:
# Finding unique values
dataset['SibSp'].unique()

In [9]:
bargraph_sibsp=sns.barplot(x='SibSp',y='Survived',data=dataset)

In [10]:
ageplot=sns.FacetGrid(dataset,col='Survived',height=8)
ageplot=ageplot.map(sns.distplot,'Age')
ageplot=ageplot.set_ylabels("Survival Probability")

In [11]:
genderplot=sns.barplot(x='Sex',y='Survived',data=dataset)

In [12]:
pclassplot=sns.catplot(x='Pclass',y='Survived',data=dataset,kind='bar',height=8)

In [13]:
bargraph_embark=sns.barplot(x='Embarked',y='Survived',data=dataset)

## DATA PREPOCESSING

In [14]:
mean=dataset['Age'].mean()
std=dataset['Age'].std()
rand_age=np.random.randint(mean-std,mean+std,size=177)
age=dataset['Age'].copy()
age[np.isnan(age)]=rand_age
dataset['Age']=age


In [15]:
dataset.Embarked.fillna(dataset.Embarked.mode()[0], inplace = True)

In [16]:
dataset.isnull().sum()

In [17]:
drop=['PassengerId','Name','Cabin','Ticket']
dataset.drop(drop,axis=1,inplace=True)
dataset.head(10)

In [18]:
genders={'male':0,'female':1}
dataset['Sex']=dataset['Sex'].map(genders)
pots={'S':0,'C':1,'Q':2}
dataset['Embarked']=dataset['Embarked'].map(pots)
dataset.head(10)

## Splitting the dataset into the Training set and Test set

In [19]:
from sklearn.model_selection import train_test_split
X=dataset.iloc[:,1:].values
y=dataset.iloc[:,0].values
x_train,x_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=42)

In [20]:
print(x_train)

In [21]:
print(y_train)

In [22]:
print(x_test)

In [23]:
print(y_test)

## Feature Scaling

In [24]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

## Random Forest

In [25]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier(random_state=42)
classifier.fit(x_train,y_train)

In [26]:
y_pred = classifier.predict(x_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
rf_acc=accuracy_score(y_test, y_pred)*100
print(rf_acc)

## Logistics Regression

In [28]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 42)
classifier.fit(x_train, y_train)

In [29]:
y_pred = classifier.predict(x_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [30]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
lr_acc=accuracy_score(y_test, y_pred)*100
print(lr_acc)

## K-Neighbour Classifier

In [31]:
from sklearn.neighbors import KNeighborsClassifier
classifier=KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
classifier.fit(x_train,y_train)

In [32]:
y_pred = classifier.predict(x_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [33]:
#test_set_confusin_matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
kn_acc=accuracy_score(y_test, y_pred)*100
print(kn_acc)

## Decision Tree

In [34]:
from sklearn.tree import DecisionTreeClassifier
classifier=DecisionTreeClassifier(criterion='entropy',random_state=42)
classifier.fit(x_train,y_train)

In [35]:
y_pred = classifier.predict(x_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [36]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
dt_acc=accuracy_score(y_test, y_pred)*100
print(dt_acc)

## Support Vector Machine

In [37]:
from sklearn.svm import SVC
classifier=SVC(kernel='linear',random_state=42)
classifier.fit(x_train,y_train)

In [38]:
y_pred = classifier.predict(x_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [39]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
svm_acc=accuracy_score(y_test, y_pred)*100
print(svm_acc)

In [40]:
print("Accuracy of Random Forest Classifier =",rf_acc)
print("Accuracy of Logistic Regressor =",lr_acc)
print("Accuracy of K-Neighbor Classifier =",kn_acc)
print("Accuracy of Decision Tree Classifier = ",dt_acc)
print("Accuracy of Support Vector Machine Classifier = ",svm_acc)

## Final Prediction

In [41]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
test_data.head(10)

In [42]:
test_data.info()

In [43]:
test_data.isnull().sum()

In [44]:
mean=test_data['Age'].mean()
std=test_data['Age'].std()
rand_age=np.random.randint(mean-std,mean+std,size=86)
age=test_data['Age'].copy()
age[np.isnan(age)]=rand_age
test_data['Age']=age
test_data['Fare'].fillna(test_data['Fare'].mean(), inplace=True)

In [45]:
col_to_drop = ["PassengerId", "Ticket", "Cabin", "Name"]
test_data.drop(col_to_drop, axis=1, inplace=True)

In [46]:
test_data.isnull().sum()

In [47]:
test_data.head(10)

In [48]:
genders={'male':0,'female':1}
test_data['Sex']=test_data['Sex'].map(genders)
pots={'S':0,'C':1,'Q':2}
test_data['Embarked']=test_data['Embarked'].map(pots)
test_data.head(10)

In [49]:
X_test = test_data
y_pred = classifier.predict(X_test)
originaltest_data = pd.read_csv('/kaggle/input/titanic/test.csv')
submission = pd.DataFrame({
        "PassengerId": originaltest_data["PassengerId"],
        "Survived": y_pred
    })
submission.head(20)

In [50]:
submission.to_csv('submission.csv',index=False)